In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import os
import sys
import transformers
import keras
import tf_keras
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, BertConfig
from tf_keras.layers import Input, Dense, Concatenate, Dropout, BatchNormalization
from tf_keras.models import Model
from tf_keras.optimizers import Adam
import math
import tensorflow_hub as hub
from sklearn.utils import resample
tf.config.run_functions_eagerly(True)
!pip install datasets
import datasets
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (1)
sst2_dataset = load_dataset("glue", "sst2")
train_dataset = sst2_dataset["train"]
validation_dataset = sst2_dataset["validation"]
test_dataset = sst2_dataset["test"]
train_df=pd.DataFrame(train_dataset.to_pandas())
val_df=pd.DataFrame(validation_dataset.to_pandas())
test_df=pd.DataFrame(test_dataset.to_pandas())
df=pd.concat([train_df, val_df], ignore_index=True)
df.drop(columns=['idx'], inplace=True)
print(df)

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (2)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['label'], random_state=42)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df, unlab_df= train_test_split(train_df, test_size=0.4, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=2
print(train_df)
print(unlab_df)
print(val_df)
print(test_df)
#For the first split, put the frac value below to be equal to 0.0031 for about 100 labeled samples
#For the second split, put the frac value below to be equal to 0.01 for about 300 labeled samples
#For the third split, put the frac value below to be equal to 0.0153 for about 500 labeled samples
#For the fourth split, put the frac value below to be equal to 0.031 for about 1000 labeled samples
#For the fifth split put the frac value below to be equal to 0.0765 for about 2500 labeled samples
#For the sixth split, put the frac value below to be equal to 0.153 for about 5000 labeled samples
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.153))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE IMDB Movie Review DATASET
cwd=os.getcwd()
data=tf.keras.utils.get_file(fname ="aclImdb.tar.gz", origin="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", cache_dir= cwd, extract = True)
data_path=os.path.join(os.path.dirname(data), 'aclImdb')
os.listdir(data_path)
train_dir=os.path.join(data_path, 'train')
for file in os.listdir(train_dir):
    file_path=os.path.join(train_dir, file)
    if os.path.isfile(file_path):
        with open(file_path) as f:
            first_line=f.readline().strip()
            print(file + ":" + first_line)
def load_dataset(dir, typeOfData):
    dictnry = {"review": [], "sentiment": []}
    if typeOfData == 'labelled':
        pos_path=os.path.join(dir, 'pos')
        neg_path=os.path.join(dir, 'neg')
        for file in os.listdir(pos_path):
            file_path=os.path.join(pos_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(1)
        for file in os.listdir(neg_path):
            file_path=os.path.join(neg_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(0)
    elif typeOfData == 'unsup':
        unsup_path=os.path.join (dir, 'unsup')
        for file in os.listdir(unsup_path):
            file_path=os.path.join(unsup_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(2)
    return dictnry
Train_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'labelled'))
Unlab_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'unsup'))
test_dir=os.path.join(data_path, 'test')
Test_df=pd.DataFrame.from_dict(load_dataset(test_dir, 'labelled'))
df=pd.concat([Train_df, Test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['sentiment'], random_state=42)
train_df, unlab_df= train_test_split(train_df, test_size=0.375, stratify=train_df['sentiment'], random_state=42)
unlab_df.loc[:, 'sentiment']=2
#For the first split, put the frac value from the train_df below to be equal to 0.004 for about 100 labeled
#For the second split, put the frac value from the train_df below to be equal to 0.012 for about 300 labeled
#For the third split, put the frac value from the train_df below to be equal to 0.04 for about 1000 labeled
#For the fourth split, put the frac value from the train_df below to be equal to 0.2 for about 5000 labeled
train_df=train_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.2))
val_df=val_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
test_df=test_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (1)
from google.colab import files
uploaded = files.upload()

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (2)
train_df=pd.read_csv('/content/Corona_NLP_train.csv', encoding='ISO-8859-1')
test_df=pd.read_csv('/content/Corona_NLP_test.csv', encoding='ISO-8859-1')
train_df=train_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
test_df=test_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
train_df.drop_duplicates(subset='OriginalTweet',inplace=True)
test_df.drop_duplicates(subset='OriginalTweet',inplace=True)
df=pd.concat([train_df, test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['Sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['Sentiment'], random_state=42)
mapping = {'Extremely Negative': 0, 'Negative': 1, 'Neutral': 2, 'Positive':3, 'Extremely Positive':4}
train_df['Sentiment']=train_df['Sentiment'].replace(mapping)
val_df['Sentiment']=val_df['Sentiment'].replace(mapping)
test_df['Sentiment']=test_df['Sentiment'].replace(mapping)
train_df['Sentiment']=train_df['Sentiment'].astype('category')
val_df['Sentiment']=val_df['Sentiment'].astype('category')
test_df['Sentiment']=test_df['Sentiment'].astype('category')
train_df, unlab_df= train_test_split(train_df, test_size=0.42, stratify=train_df['label'], random_state=42)
unlab_df['Sentiment']=unlab_df['Sentiment'].cat.add_categories(5)
unlab_df['Sentiment']=5
#For the first split, put the frac value below to be equal to 0.0051
#For the second split, put the frac value below to be equal to 0.0255
#For the third split, put the frac value below to be equal to 0.051
#For the fourth split, put the frac value below to be equal to 0.2545
train_df=train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.0051))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT SST-5 DATASET
dataset=load_dataset("SetFit/sst5")
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']
train_df=pd.DataFrame(train_dataset)
train_df=train_df.drop(columns=['label_text'])
val_df=pd.DataFrame(val_dataset)
val_df=val_df.drop(columns=['label_text'])
test_df=pd.DataFrame(test_dataset)
test_df=test_df.drop(columns=['label_text'])
train_df, unlab_df= train_test_split(train_df, test_size=0.53, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=5
#For the first split, put the frac value below to be equal to 0.027
#For the second split, put the frac value below to be equal to 0.125
#For the third split, put the frac value below to be equal to 0.25
#For the fourth split, put the frac value below to be equal to 0.5
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.125))
train_df.drop_duplicates(subset='text',inplace=True)
unlab_df.drop_duplicates(subset='text', inplace=True)
val_df.drop_duplicates(subset='text',inplace=True)
test_df.drop_duplicates(subset='text',inplace=True)
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
def clean_text(text):
    soup=BeautifulSoup(text, "html.parser")
    text=re.sub(r"\[[^]]*\]" , '', soup.get_text())
    no_url=re.sub(r'http\S+','', text)
    lowerCase=no_url.lower()
    new_text=re.sub(r"[^a-zA-Z\s,'#]", '', lowerCase)
    new_text=re.sub(r'[\r\n\t]', '', new_text)
    new_text=re.sub(r'#\S*', '', new_text)
    return new_text

In [ ]:
train_df['Cleaned']=train_df[train_df.columns[0]].apply(clean_text)
unlab_df['Cleaned']=unlab_df[train_df.columns[0]].apply(clean_text)
val_df['Cleaned']=val_df[train_df.columns[0]].apply(clean_text)
test_df['Cleaned']=test_df[train_df.columns[0]].apply(clean_text)

In [ ]:
trainX=train_df['Cleaned']
trainY=train_df[train_df.columns[1]]
unlabX=unlab_df['Cleaned']
unlabY=unlab_df[train_df.columns[1]]
valX=val_df['Cleaned']
valY=val_df[train_df.columns[1]]
testX=test_df['Cleaned']
testY=test_df[train_df.columns[1]]
num_classes=len(trainY.unique())

In [ ]:
class MLPGenerator(tf.keras.Model):
    def __init__(self, layer_size=768, output_size=768):
        super().__init__()
        self.dense1=Dense(units=layer_size)
        self.activation1=tf_keras.layers.LeakyReLU()
        self.drop1=Dropout(0.1)
        self.output_layer=tf_keras.layers.Dense(units=output_size, activation=None)
    def call(self, input, training=True):
        x=self.dense1(input)
        x=self.activation1(x)
        x=self.drop1(x, training=training)
        self.x=self.output_layer(x)
        return self.x
    def gen_loss(self, real_activations, fake_activations, d_fake_outputs, smooth=0.1):
        feature_matching=tf.reduce_mean(tf.square(tf.reduce_mean(real_activations, axis=0) - tf.reduce_mean(fake_activations, axis=0)))
        gUnsupLoss=-tf.reduce_mean(tf.math.log(1-d_fake_outputs[:,-1]+1e-10))
        self.genLoss=feature_matching + gUnsupLoss
        return self.genLoss

In [ ]:
class MLPDiscriminator(tf.keras.Model):
    def __init__(self, layer_size=768, output_size=num_classes+1):
        super().__init__()
        self.bert_model=TFBertModel.from_pretrained('bert-base-uncased')
        self.dense1=tf_keras.layers.Dense(units=layer_size)
        self.activation1=tf_keras.layers.LeakyReLU()
        self.drop1=Dropout(0.1)
        self.dense2=tf_keras.layers.Dense(units=output_size, activation=None)
    def call(self, inputs, training=True):
        if dataType == 'R':
            inputIds, attentionMask= inputs
            embeddings=self.bert_model(inputIds, attentionMask)['last_hidden_state'][:,0]
        elif dataType == 'F':
            embeddings=inputs
        x=self.dense1(embeddings)
        self.activations=self.activation1(x)
        x=self.drop1(x, training=training)
        self.logitsD=self.dense2(x)
        self.d_output=tf.nn.softmax(self.logitsD, axis=-1)
        return self.activations, self.logitsD, self.d_output

    def disc_loss(self, d_real_logits, d_real_outputs, targets, training, d_fake_outputs=None, label_mask=None):
        if training:
            if tf.reduce_any(tf.not_equal(targets, num_classes)).numpy() == 0:
                disc_supLoss=0
            else:
                masked_real_logits=tf.boolean_mask(d_real_logits, label_mask)
                if num_classes == 2:
                    real_probs=tf.nn.sigmoid(masked_real_logits[:,0:-1])
                    log_probs=tf.math.log(real_probs)
                elif num_classes == 5:
                    log_probs=tf.nn.log_softmax(masked_real_logits[:,0:-1])
                masked_targets=tf.boolean_mask(targets, label_mask)
                one_hot_labels=tf.one_hot(masked_targets, num_classes)
                per_data_sup_loss=-tf.reduce_sum(one_hot_labels*log_probs)
                disc_supLoss=tf.divide(per_data_sup_loss, tf.maximum(1.0, tf.cast(log_probs.shape[0], tf.float32)))
            unlab_mask=tf.zeros(d_real_outputs.shape[0])
            for i in range(d_real_outputs.shape[0]):
                if targets[i] != num_classes and tf.argmax(d_real_outputs[i]) == num_classes:
                    unlab_mask=tf.tensor_scatter_nd_update(unlab_mask, [[i]], [1])
            masked_real_outputs=tf.boolean_mask(d_real_outputs, unlab_mask)
            if masked_real_outputs.shape[0] == 0:
                real_unsup_loss=0.0
            else:
                real_unsup_loss=-tf.reduce_mean(tf.math.log(1-masked_real_outputs[:,-1])+1e-10)
            fake_unsup_loss=-tf.reduce_mean(tf.math.log(1-d_fake_outputs[:,-1])+1e-10)
            disc_unsupLoss=real_unsup_loss+fake_unsup_loss
            discLoss=disc_supLoss+disc_unsupLoss
            return discLoss
        else:
            one_hot_labels=tf.one_hot(targets, num_classes)
            if num_classes == 2:
                real_probs=tf.nn.sigmoid(d_real_logits[:,0:-1])
                log_probs=tf.math.log(real_probs)
            elif num_classes == 5:
                log_probs=tf.nn.log_softmax(d_real_logits[:,0:-1])
            per_data_sup_loss=-tf.reduce_sum(one_hot_labels*log_probs)
            disc_supLoss=tf.reduce_mean(per_data_sup_loss)
            discLoss=disc_supLoss
            return discLoss

In [ ]:
class GANBERT(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.gen=MLPGenerator()
        self.disc=MLPDiscriminator()
        self.tz=BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.bert_model = TFBertModel.from_pretrained('bert-base-uncased', num_attention_heads=12)
        self.previous_supervised_accuracy=0.0
        self.previous_unsupervised_accuracy=0.0

    def encode_text(self, input_text, max_length=64):
        self.max_length=max_length
        tokens=self.tz.batch_encode_plus(input_text.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
        return tokens

    def preparing_dataset(self, trainX, trainY, unlabX, unlabY, valX, valY, testX, testY, log_ratio=int(math.log(100, 10))):
        self.trainX=trainX
        self.trainY=trainY
        self.unlabX=unlabX
        self.unlabY=unlabY
        self.valX=valX
        self.valY=valY
        self.testX=testX
        self.testY=testY
        self.trainX_embeddings=self.encode_text(self.trainX)
        print(self.trainX_embeddings['input_ids'].shape)
        with tf.device('/CPU:0'):
            self.trainX_embeddings['input_ids']=tf.repeat(self.trainX_embeddings['input_ids'], repeats=log_ratio, axis=0)
            self.trainX_embeddings['attention_mask']=tf.repeat(self.trainX_embeddings['attention_mask'], repeats=log_ratio, axis=0)
            self.trainY=tf.repeat(self.trainY, repeats=log_ratio, axis=0)
        self.trainX_embeddings['input_ids']=self.trainX_embeddings['input_ids']
        print(self.trainX_embeddings['input_ids'].shape)
        self.trainX_embeddings['attention_mask']=self.trainX_embeddings['attention_mask']
        self.trainY=self.trainY
        self.unlabX_embeddings=self.encode_text(self.unlabX)
        with tf.device('/CPU:0'):
            bertX_inputIds=tf.concat([self.trainX_embeddings['input_ids'], self.unlabX_embeddings['input_ids']], axis=0)
            bertX_attentionMask=tf.concat([self.trainX_embeddings['attention_mask'], self.unlabX_embeddings['attention_mask']], axis=0)
            self.berty=tf.concat([self.trainY, self.unlabY.to_numpy()], axis=0)
        print(bertX_inputIds.shape)
        self.berty=self.berty
        self.berty=tf.cast(self.berty, tf.int32)
        self.valX=self.encode_text(valX)
        self.testX=self.encode_text(testX)
        num_samples=bertX_inputIds.shape[0]
        random_indexes=tf.random.shuffle(tf.range(num_samples))
        with tf.device('/CPU:0'):
            bertX_inputIds=tf.gather(bertX_inputIds, random_indexes)
            bertX_attentionMask=tf.gather(bertX_attentionMask, random_indexes)
            self.berty=tf.gather(self.berty, random_indexes)
        self.bertX=(bertX_inputIds, bertX_attentionMask)
        print(self.bertX[0].shape)
        self.berty=self.berty
        self.valX=(self.valX['input_ids'], self.valX['attention_mask'])
        self.valY=tf.convert_to_tensor(self.valY)
        self.valY=self.valY
        self.testX=(self.testX['input_ids'], self.testX['attention_mask'])
        self.testY=tf.convert_to_tensor(self.testY)
        self.testY=self.testY
        return self.bertX, self.berty, self.valX, self.valY, self.testX, self.testY

    def call(self, real_data, training=True):
        global dataType
        if training == True:
            self.gout=self.gen(self.fake_batch)
            dataType='R'
            self.real_activs, self.real_logits, self.real_outputs=self.disc(real_data, training=training)
            dataType='F'
            self.fake_activs, self.fake_logits, self.fake_outputs=self.disc(self.gout, training=training)
            return self.gout, self.real_activs, self.real_logits, self.real_outputs, self.fake_activs, self.fake_logits, self.fake_outputs
        else:
            dataType='R'
            test_activs, test_logits, test_outputs=self.disc(real_data, training=training)
            return test_activs, test_logits, test_outputs

    def compile(self, gen_opt, disc_opt, sup_acc, unsup_acc, run_eagerly=True):
        super().compile()
        self.gen_opt=gen_opt
        self.disc_opt=disc_opt
        self.sup_acc=sup_acc
        self.unsup_acc=unsup_acc
        return self.gen_opt, self.disc_opt, self.sup_acc, self.unsup_acc

    def train_step(self, data):
        real_batch, real_targets=data
        fake_batch_size=32 #YOU SET THE FAKE BATCH SIZE
        self.fake_batch=tf.random.normal(mean=0, stddev=1, shape=(fake_batch_size, 100))
        self.fake_targets=tf.constant([2]*fake_batch_size)
        label_mask=tf.zeros(real_batch[0].shape[0])
        count=0
        real_unlabeled_idx=[]
        for j in real_targets:
            if j != num_classes:
                label_mask=tf.tensor_scatter_nd_update(label_mask, [[count]], [1])
            else:
                real_unlabeled_idx.append(count)
            count=count+1
        with tf.GradientTape() as gen_tape:
            gout, real_activs, real_logits, real_outputs, fake_activs, fake_logits, fake_outputs=self.call(real_batch, training=True)
            gen_loss=self.gen.gen_loss(real_activs, fake_activs, fake_outputs)
        gradients_gen=gen_tape.gradient(gen_loss, self.gen.trainable_variables)
        self.gen_opt.apply_gradients(zip(gradients_gen, self.gen.trainable_variables))
        with tf.GradientTape() as disc_tape:
            gout, real_activs, real_logits, real_outputs, fake_activs, fake_logits, fake_outputs=self.call(real_batch, training=True)
            disc_loss=self.disc.disc_loss(real_logits, real_outputs, real_targets, True, fake_outputs, label_mask)
        gradients_disc=disc_tape.gradient(disc_loss, self.disc.trainable_variables)
        self.disc_opt.apply_gradients(zip(gradients_disc, self.disc.trainable_variables))
        masked_real_outputs=tf.boolean_mask(real_outputs, label_mask)
        masked_real_targets=tf.boolean_mask(real_targets, label_mask)
        if masked_real_targets.shape[0] != 0:
            preds=masked_real_outputs[:,0:-1]
            masked_real_targets=tf.one_hot(masked_real_targets, num_classes)
            self.sup_acc.update_state(masked_real_targets, preds)
            self.previous_supervised_accuracy = self.sup_acc.result().numpy()
        if len(real_unlabeled_idx) != 0:
            real_unlabeled_targets=tf.ones_like(tf.gather(real_targets, real_unlabeled_idx))
            fake_unlabeled_targets=tf.zeros_like(self.fake_targets)
            real_unlabeled_logits=tf.gather(real_logits, real_unlabeled_idx)
            unlabeled_logits=tf.concat([real_unlabeled_logits, fake_logits], axis=0)
            unlabeled_targets=tf.concat([real_unlabeled_targets, fake_unlabeled_targets], axis=0)
            randomIdx=tf.random.shuffle(tf.range(unlabeled_targets.shape[0]))
            unlabeled_targets=tf.gather(unlabeled_targets, randomIdx)
            unlabeled_logits=tf.gather(unlabeled_logits, randomIdx)
            probs=tf.nn.sigmoid(unlabeled_logits[:,-1])
            self.unsup_acc.update_state(tf.cast(unlabeled_targets, tf.float32), probs)
            self.previous_unsupervised_accuracy = self.unsup_acc.result().numpy()
        return {"gen_loss": gen_loss.numpy(), "disc_loss": disc_loss.numpy(), "supervised accuracy":self.sup_acc.result().numpy() if masked_real_targets.shape[0] != 0 else self.previous_supervised_accuracy, "unsupervised accuracy":self.unsup_acc.result().numpy() if len(real_unlabeled_idx) != 0 else self.previous_unsupervised_accuracy}
    def test_step(self, data):
        samples, targets=data
        test_activs, test_logits, test_outputs=self.call(samples, training=False)
        loss=self.disc.disc_loss(test_logits, test_outputs, targets, training=False)
        pred_test=test_outputs[:,0:-1]
        targets=tf.one_hot(targets, num_classes)
        self.sup_acc.update_state(targets, pred_test)
        return {"loss": loss.numpy(), "accuracy": self.sup_acc.result().numpy()}

In [ ]:
test=GANBERT()

In [ ]:
semi=test.preparing_dataset(trainX, trainY, unlabX, unlabY, valX, valY, testX, testY)

In [ ]:
gen_optimizer = tf_keras.optimizers.Adam(learning_rate=1e-5)
disc_optimizer = tf_keras.optimizers.Adam(learning_rate=1e-5)
if num_classes == 2:
    supervised_accuracy = tf_keras.metrics.BinaryAccuracy('supervised accuracy')
elif num_classes == 5:
    supervised_accuracy = tf_keras.metrics.CategoricalAccuracy('supervised accuracy')
unsupervised_accuracy = tf_keras.metrics.BinaryAccuracy('unsupervised accuracy')
test.compile(gen_optimizer, disc_optimizer, supervised_accuracy, unsupervised_accuracy)

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((semi[0], semi[1])).shuffle(semi[0][0].shape[0]).batch(32)
val_dataset=tf.data.Dataset.from_tensor_slices((semi[2], semi[3])).shuffle(semi[2][0].shape[0]).batch(32)
test_dataset=tf.data.Dataset.from_tensor_slices((semi[4], semi[5])).shuffle(semi[4][0].shape[0]).batch(32)

In [ ]:
for epoch in range(3):
    test.reset_metrics()
    history=test.fit(train_dataset, batch_size=32, epochs=1, shuffle=True)
    test.reset_metrics()
    test.evaluate(val_dataset)

In [ ]:
test.reset_metrics()
results = test.evaluate(test_dataset)

In [ ]:
#FROM HERE STARTS THE SUPERVISED PART, RUN ALL THE CELLS BELOW
train_df['Cleaned']=train_df[train_df.columns[0]].apply(clean_text)
val_df['Cleaned']=val_df[train_df.columns[0]].apply(clean_text)
test_df['Cleaned']=test_df[train_df.columns[0]].apply(clean_text)

In [ ]:
trainX=train_df['Cleaned']
trainY=tf.convert_to_tensor(train_df[train_df.columns[1]])
valX=val_df['Cleaned']
valY=tf.convert_to_tensor(val_df[val_df.columns[1]])
testX=test_df['Cleaned']
testY=tf.convert_to_tensor(test_df[test_df.columns[1]])

In [ ]:
max_length=64
tz=BertTokenizer.from_pretrained('bert-base-uncased')
trainX_encoded=tz(trainX.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
valX_encoded=tz(valX.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
testX_encoded=tz(testX.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

In [ ]:
def custom_loss(y_true, y_pred_logits, num_classes=num_classes):
    log_probs=tf.nn.log_softmax(y_pred_logits[:,0:num_classes])
    per_data_sup_loss=-tf.reduce_sum(y_true*log_probs)
    loss=tf.reduce_mean(per_data_sup_loss)
    return loss

In [ ]:
trainY=tf.one_hot(trainY, num_classes)
valY=tf.one_hot(valY, num_classes)
testY=tf.one_hot(testY, num_classes)

In [ ]:
opt=tf_keras.optimizers.Adam(learning_rate=3e-5)
callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=5e-4, patience=10, restore_best_weights=True)
if num_classes == 2:
    metric=tf_keras.metrics.BinaryAccuracy('accuracy')
elif num_classes == 5:
    metric=tf_keras.metrics.CategoricalAccuracy('accuracy')
num_epochs=100
config=BertConfig.from_pretrained("bert-base-uncased")
dropout_rate=0.3
config.hidden_dropout_prob=dropout_rate
config.num_labels=num_classes
model=TFBertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)
model.compile(optimizer=opt, loss=custom_loss, metrics=[metric])

In [ ]:
history=model.fit([trainX_encoded['input_ids'], trainX_encoded['attention_mask']], trainY, batch_size=32, epochs=num_epochs, verbose=1, shuffle=True, callbacks=[callback], validation_data=([valX_encoded['input_ids'], valX_encoded['attention_mask']], valY))

In [ ]:
test_loss, test_accuracy = model.evaluate([testX_encoded['input_ids'], testX_encoded['attention_mask']], testY)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')